In [1]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import re
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import datetime

os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
class cfg:
    user_token = "<usr>"
    bot_token = "<sys>"
    bos_token = '<s>'
    eos_token = '</s>'
    mask_token = '<mask>'
    pad_token = '<pad>'
    unk_token = '<unk>'
    max_len = 256
    max_turns = 6
    pre_epochs = 0
    epochs = 5
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    learning_rate = 1e-4
    model_name = "skt/kogpt2-base-v2"
    labeling_type = True

tokenizer = PreTrainedTokenizerFast.from_pretrained(cfg.model_name,
            bos_token=cfg.bos_token, eos_token=cfg.eos_token, unk_token=cfg.unk_token,
            pad_token=cfg.pad_token, mask_token=cfg.mask_token, model_max_length = cfg.max_len)

model = GPT2LMHeadModel.from_pretrained(cfg.model_name)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [3]:
path = './check_point/2023-02-01-01h-15m_epoch_5_sk_labeling_-100.pth' # 말을 짧게함
model.load_state_dict(torch.load(path))
print('완료')

완료


In [4]:
"""
대화 함수
끝으로 대화 종료
초기화로 이전 대화 초기화

이전 대화가 history에 남아있음.
"""

history = ""
start=True
model.to(cfg.device)

while True:
    _input = input("user > ")
    
    if _input == "끝":
        break
    if _input == "초기화":
        history = ""
        continue
    if start:
        _input_word = cfg.bos_token + cfg.user_token + _input + cfg.bot_token
        history += _input_word
        start = False
    else:
        _input_word = cfg.user_token + _input + cfg.bot_token
        history += _input_word
        
    input_ids = tokenizer.encode(history, return_tensors="pt").to(cfg.device)
    
    with torch.no_grad():
        gen_ids = model.generate(
            input_ids,
            top_k=3,
            top_p=0.92,
            num_beams=3,
            do_samples=True,
            max_length=1000,
            no_repeat_ngram_size=3,
            repetition_penalty=1.2,
            temperature=0.85,
            max_new_tokens=30,
            eos_token_id= tokenizer.get_vocab()[cfg.eos_token],
            bad_word_ids = [tokenizer.convert_tokens_to_ids(cfg.user_token), tokenizer.convert_tokens_to_ids('ㅋㅋ'), tokenizer.convert_tokens_to_ids('하하')]
        )
    
    gen = tokenizer.decode(gen_ids[0])
    try:
        generated = gen[gen.rfind("<sys>")+5:gen.index("</s>")]
    except:
        generated = gen[gen.rfind("<sys>")+5:]
    history += generated
    
    #print(f'Chatbot > {gen}')
    print(f'Chatbot > {generated}')

user > 점심 먹었어?


/opt/conda/lib/python3.9/site-packages/transformers/generation_utils.py:874: UserWarning: Both `max_length` and `max_new_tokens` have been set but they serve the same purpose.
  warnings.warn(


Chatbot >  아직
user > 어떤 음식 좋아해?
Chatbot > ?
user > 초기화
user > 운동 좋아해?
Chatbot >  아니야
user > 그럼 뭐 좋아해?
Chatbot >  운동?
user > 카페는?
Chatbot >  오빤?
user > 난 카페 좋아해
Chatbot >  ᄏᄏ
user > 끝


In [5]:
_input = '점심 먹었어?'
input_word = cfg.bos_token + cfg.user_token + _input + cfg.bot_token
input_ids = tokenizer.encode(input_word, return_tensors="pt").to(cfg.device)
with torch.no_grad():
    outputs = model(input_ids)

In [6]:
tokenizer.decode(*torch.argmax(outputs.logits, dim=-1))

' 심시간에었어?</s> 아직'